In [ ]:
import pandas as pd
import sqlite3
def load_csv_to_sqlite(db_name, csv_files):
conn = sqlite3.connect(db_name)
for table_name, file_path in csv_files.items():
df = pd.read_csv(file_path)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()
csv_files = {
'ideal': 'ideal.csv',
'test': 'test.csv',
'train': 'train.csv'
}
db_name = 'datasets.db'
load_csv_to_sqlite(db_name, csv_files)

In [ ]:
def load_data_from_sqlite(db_name):
conn = sqlite3.connect(db_name)
Page 6 of 15 - AI Writing Submission Submission ID trn:oid:::1:2963661562
Page 6 of 15 - AI Writing Submission Submission ID trn:oid:::1:2963661562
ideal_df = pd.read_sql('SELECT * FROM ideal', conn)
test_df = pd.read_sql('SELECT * FROM test', conn)
train_df = pd.read_sql('SELECT * FROM train', conn)
conn.close()
return ideal_df, test_df, train_df
ideal_df, test_df, train_df = load_data_from_sqlite(db_name)

In [ ]:
from sklearn.metrics import mean_squared_error
def find_best_fit(train_df, ideal_df):
best_fit_functions = []
for train_col in train_df.columns[1:]:
best_fit = None
lowest_mse = float('inf')
for ideal_col in ideal_df.columns[1:]:
mse = mean_squared_error(train_df[train_col], ideal_df[ideal_col])
if mse < lowest_mse:
lowest_mse = mse
best_fit = ideal_col
best_fit_functions.append((train_col, best_fit, lowest_mse))
return best_fit_functions
best_fit_functions = find_best_fit(train_df, ideal_df)

In [ ]:
def map_test_data(test_df, ideal_df, best_fit_functions):
mappings = []
for index, row in test_df.iterrows():
x = row['x']
y = row['y']
for train_col, ideal_col, _ in best_fit_functions:
ideal_y = ideal_df.loc[ideal_df['x'] == x, ideal_col].values[0]
deviation = abs(y - ideal_y)
mappings.append((x, y, train_col, ideal_col, deviation))
return pd.DataFrame(mappings, columns=['x', 'y', 'train_col', 'ideal_col', 'deviation'])
mappings_df = map_test_data(test_df, ideal_df, best_fit_functions)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
def visualize_data_bokeh(train_df, ideal_df, test_df, mappings_df, best_fit_functions):
plots = []
for train_col, ideal_col, _ in best_fit_functions:
p = figure(title=f'{train_col} and {ideal_col}', x_axis_label='x', y_axis_label='y')
p.line(train_df['x'], train_df[train_col], legend_label=f'Train {train_col}', line_color="blue")
Page 8 of 15 - AI Writing Submission Submission ID trn:oid:::1:2963661562
Page 8 of 15 - AI Writing Submission Submission ID trn:oid:::1:2963661562
p.line(ideal_df['x'], ideal_df[ideal_col], legend_label=f'Ideal {ideal_col}', line_color="green")
mapped_data = mappings_df[mappings_df['train_col'] == train_col]
p.scatter(mapped_data['x'], mapped_data['y'], legend_label='Test Data', color='red')
plots.append(p)
grid = gridplot([plots], ncols=2, plot_width=400, plot_height=400)
output_notebook()
show(grid)
visualize_data_bokeh(train_df, ideal_df, test_df, mappings_df, best_fit_functions)